In [1]:
import pandas as pd
import json
data = {"username":[], "classes":[]}
# Opening JSON file
with open('DATA/user_categories.json', 'r') as openfile:
    json_object = json.load(openfile)
    for user, subs in json_object:
        if(len(subs) > 0):
            data["username"].append(user)
            data["classes"].append(subs)

df = pd.DataFrame(data)
df

,username,classes
0,jkcc,F F I A
1,wsxmm1153,F G G D
2,ksax,B B G G I B B B B B E F F F B B D
3,oddur09,A D F F
4,harryapotter,D C F F B I B B D F D A G
...,...,...
67139,nparrado,F F G D C B D D F B H B C B D F G
67140,Shampoo,F F F F F F F F F F G G G G D D D D D D D D D ...
67141,gaston770,F D A H D D A I I I I A I I I I A B E I C C D ...
67142,mandycyt,D B I C D B


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer="char", preprocessor= lambda x : x.replace(" ", "")) # TfIdf
vector_matrix = count_vectorizer.fit_transform(list(df["classes"]))
vector_matrix

<67144x9 sparse matrix of type '<class 'numpy.int64'>'
	with 328095 stored elements in Compressed Sparse Row format>

In [3]:
vector_matrix = pd.DataFrame(vector_matrix.todense())
vector_matrix

,0,1,2,3,4,5,6,7,8
0,1,0,0,0,0,2,0,0,1
1,0,0,0,1,0,1,2,0,0
2,0,9,0,1,1,3,2,0,1
3,1,0,0,1,0,2,0,0,0
4,1,3,1,3,0,3,1,0,1
...,...,...,...,...,...,...,...,...,...
67139,0,4,2,4,0,4,2,1,0
67140,190,706,335,484,215,364,506,342,338
67141,8,24,10,21,9,8,11,7,11
67142,0,2,1,2,0,0,0,0,1


In [4]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

user_descriptions = [s.split() for s in df["classes"]]

# model = Word2Vec(sentences=user_descriptions, vector_size=100, window=5, min_count=1, sg=0)

user_embeddings = vector_matrix

# K-means
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_assignments = kmeans.fit_predict(user_embeddings)

df["cluster"] = cluster_assignments
df

,username,classes,cluster
0,jkcc,F F I A,0
1,wsxmm1153,F G G D,0
2,ksax,B B G G I B B B B B E F F F B B D,0
3,oddur09,A D F F,0
4,harryapotter,D C F F B I B B D F D A G,0
...,...,...,...
67139,nparrado,F F G D C B D D F B H B C B D F G,0
67140,Shampoo,F F F F F F F F F F G G G G D D D D D D D D D ...,2
67141,gaston770,F D A H D D A I I I I A I I I I A B E I C C D ...,7
67142,mandycyt,D B I C D B,0


In [5]:
vector_matrix["cluster"] = cluster_assignments

In [6]:
vector_matrix["cluster"].value_counts()

0    57283
7     7094
5     2106
9      508
4      105
2       31
1       10
8        4
6        2
3        1
Name: cluster, dtype: int64

In [7]:
df[df["cluster"] == 7]

,username,classes,cluster
8,mario88,F G D D I D D D I E A A G I I D G I C I A B H ...,7
13,a1303,F F F F F F F F F F F F F F F D D G B G A B D ...,7
25,whitesoftsweet,G G G G B I I D B I G I D B D E G E B H H F D ...,7
26,zloig,F F F F F F F F F F F F F F F F F F F F F F F ...,7
41,MarcoRoZa,F D A A G I I I D G G H D D D C I A E E E E H ...,7
...,...,...,...
67118,vanius,I I I I I I I I I I I I I I I I I H H H H H H ...,7
67123,windtw,F D I F I C D B I D A A F B B B H B B D B G B ...,7
67125,katotemp,F F F F F D I I I A I G H H C C G C H H H D C ...,7
67134,kublai1228,F F I C G I D B E H H B D A A H F B I B H G H ...,7
